In [1]:
import pandas as pd
import numpy as np

<h2>
Reading and processing data
</h2>

In [2]:
slab = pd.read_csv('data/slab.csv', sep=";")
defect = pd.read_excel('data/defect.xlsx')
heat = pd.read_excel('data/heat.xlsx')
process = pd.read_csv('data/process.csv', sep=";")

In [3]:
LF = pd.read_excel('data/LF.xlsx')
EAF = pd.read_excel('data/eaf.xlsx')

In [4]:
CCM = pd.read_excel('data/ccm.xlsx')

In [5]:
eaf_ladels = pd.DataFrame(EAF["LADEL"].unique())
lf_ladels = pd.DataFrame(LF["LADEL"].unique())
ccm_ladels = pd.DataFrame(CCM["LADEL"].unique())
ladels = pd.factorize(pd.concat([eaf_ladels, lf_ladels, ccm_ladels])[0].unique(), na_sentinel=None)
ladels_replacement = {}
for i in range(len(ladels[0])): ladels_replacement[ladels[1][i]] = ladels[0][i]

tundish = pd.factorize(CCM["TUNDISH"].unique())
tundish_replacements = {}
for i in range(len(tundish[0])): tundish_replacements[tundish[1][i]] = tundish[0][i]

shells = pd.factorize(EAF["SHELL"].unique())
shells_replacements = {}
for i in range(len(shells[0])): shells_replacements[shells[1][i]] = shells[0][i]

eaf_itmdfs = pd.DataFrame(EAF["ITMDF_COD_ITM_ITMDF"].unique())
lf_itmdfs = pd.DataFrame(LF["ITMDF_COD_ITM_ITMDF"].unique())
ccm_itmdfs = pd.DataFrame(CCM["ITMDF_COD_ITM_ITMDF"].unique())
itmdfs = pd.factorize(pd.concat([eaf_itmdfs, lf_itmdfs, ccm_itmdfs])[0].unique(), na_sentinel=None)
itmdfs_replacement = {}
for i in range(len(itmdfs[0])): itmdfs_replacement[(itmdfs[1][i])] = itmdfs[0][i]

# replacing descriptive data with numric data
LF["LADEL"].replace(ladels_replacement, inplace=True)
EAF["LADEL"].replace(ladels_replacement, inplace=True)
CCM["LADEL"].replace(ladels_replacement, inplace=True)

CCM["TUNDISH"].replace(tundish_replacements, inplace=True)
EAF["SHELL"].replace(shells_replacements, inplace=True)

EAF["ITMDF_COD_ITM_ITMDF"].replace(itmdfs_replacement, inplace=True)
LF["ITMDF_COD_ITM_ITMDF"].replace(itmdfs_replacement, inplace=True)
CCM["ITMDF_COD_ITM_ITMDF"].replace(itmdfs_replacement, inplace=True)

In [6]:
# removing VAL_ITM_A_PRITM (too many nans)
CCM = CCM.drop('VAL_ITM_A_PRITM', axis=1)
EAF = EAF.drop('VAL_ITM_A_PRITM', axis=1)
LF = LF.drop('VAL_ITM_A_PRITM', axis=1)

In [7]:
# from sklearn.experimental import enable_iterative_imputer
# from sklearn.impute import IterativeImputer

# imp = IterativeImputer(max_iter=500, random_state=0)
# imp.fit(np.array(CCM))
# CCM = np.round(imp.transform(np.array(CCM)), 0)

# imp.fit(np.array(EAF))
# EAF = np.round(imp.transform(np.array(EAF)), 0)

# imp.fit(np.array(LF))
# LF = np.round(imp.transform(np.array(LF)), 0)

In [8]:
merge1 = pd.merge(slab, defect, how="outer", left_on='PRODUCT_ID', right_on='PRODT_PRODUCT_ID')
merge1 = pd.merge(merge1, heat, how="outer", left_on='PRODT_PRODUCT_ID_x', right_on='PRODUCT_ID')

ccm_process = pd.merge(process, CCM, how="outer", left_on='PROCESS_RESULT_ID', right_on='PCRSL_PROCESS_RESULT_ID')
lf_process = pd.merge(process, LF, how="outer", left_on='PROCESS_RESULT_ID', right_on='PCRSL_PROCESS_RESULT_ID')
eaf_process = pd.merge(process, EAF, how="outer", left_on='PROCESS_RESULT_ID', right_on='PCRSL_PROCESS_RESULT_ID')
processes = pd.concat([ccm_process, lf_process, eaf_process])

In [14]:
processes = processes.drop('PROCESS_RESULT_ID', axis=1)


KeyError: "['PROCESS_RESULT_ID'] not found in axis"

In [15]:
print(len(processes))

4514878


In [10]:
dataset = pd.merge(merge1, processes, how="outer", left_on='PRODT_PRODUCT_ID_x', right_on='PRODT_PRODUCT_ID')